# Repetition codes

In this notebook we'll cover the following topics:

 - Binary encodings
 - Repetition codes

## Binary encodings

As ten-fingered humans, we naturally gravitate towards base-ten units, which is evident in our currency and scientific measurements (though, interestingly, not in timekeeping). Early computational devices, such as abacuses with rows of ten beads, followed this trend. The first mechanical computers, like the Difference Engine (https://en.wikipedia.org/wiki/Difference_engine), also employed ten-position dials. While these devices were somewhat effective, they lacked scalability due to the engineering challenge of creating precise dials. Manufacturing faults could lead to errors that propagated throughout the system, negatively impacting computational outcomes.

By the late 1930s, most computers transitioned to binary encoding, utilizing sequences of '0's and '1's to represent information. The primary advantage of binary encoding is that bits, the fundamental computational units, correspond to the on/off states of switches. Switches are among the simplest mechanical systems to engineer, making binary computers highly resilient to errors. Consequently, the majority of contemporary computing systems are built on binary logic.

## Decimal-to-Binary Mappings

In binary representation, numbers are expressed using only two digits: $0$ and $1$. Each digit in a binary number represents a power of $2$, starting from the right with $2^0$, then moving to the left with $2^1$, $2^2$, and so on. To convert a decimal number to binary, we can represent it as a sum of powers of $2$, using only the digits $0$ and $1$.

For example, the number number $5$ can be decomposed into the following sum:

$$5=\alpha_2 2^2 + \alpha_12^1 + \alpha_0 2^0$$

where $\alpha_2 = 1$, $\alpha_1 = 0$ and and $\alpha_0 = 1$. The binary representation of $5$ is given by:

$$ {\text{bin}}(5) = \left[\alpha_2, \alpha_1, \alpha_0\right] = \left[1,0,1\right]$$

In general, the length of a binary string $l$ required to represent a decimal number $x$ is given by:

$$ l = \left\lceil \log_2{(x+1)} \right\rceil $$

where $\left\lceil . \right\rceil$ is the round-up operator. Eg. using numpy we can compute this as follows:

```python
l = np.ceil(np.log2(x+1))
```

The first $9$ binary strings are shown in the table below:

| Decimal | Binary | ⌈log2(x+1)⌉ |
|---------|--------|--------------|
| 0       | 0      | 0            |
| 1       | 1      | 1            |
| 2       | 10     | 2            |
| 3       | 11     | 2            |
| 4       | 100    | 3            |
| 5       | 101    | 3            |
| 6       | 110    | 3            |
| 7       | 111    | 3            |
| 8       | 1000   | 4            |
| 9       | 1001   | 4            |


### Exercise 1.00

Write a function that converts a decimal number into a binary numpy array.
 - I have written a skeleton for this function in `src/hwbsc/binary.py`.
 - I have written the first test in `tests/test_binary.py`. Write some more.
 - Make sure to add docstrings!

Feel free to use any resources available to you: other group members, Stackoverflow, ChatGPT etc etc. However, make sure you understand the final code. Write comments on each line to help understanding and readability.

### Exercise 1.01

Write a function that converts a `numpy` binary vector to a decimal number (ie. the reverse of the last function).
 - See skeleton in `src/hwbsc/binary.py`
 - Add tests to `tests/test_binary.py`
 - Make sure to document etc.


## Binary encodings and and the logical *error distance*

Now that we can convert decimal numbers to binary numbers we can test how resilient binary encodings are to error. Use your new `hwbsc.binary.decimal2binary` function to compute the binary encoding of the number $42$. You should get this:

In [9]:
import numpy as np
binary42 = np.array([1, 0, 1, 0, 1, 0], dtype=int)

Now, what happens if this binary string is subject to a single bit-flip on one of its bits. We can investigate this affect by adding a weight 1 vector modulo 2 to the binary encoding:

In [10]:
binaryError = np.array([0, 0, 0, 0, 1, 0], dtype=int)
# we use the %2 operator to stipulate that the vectors are being added
# modulo 2. This is because the vectors are binary vectors.
ErroredBinary42 = (binary42 + binaryError) %2
print("Corrupted binary encoding", ErroredBinary42)

Corrupted binary encoding [1 0 1 0 0 0]


Ok, now our binary encoding is corrupted.
It we use the `hwbsc.binary.binary2decimal` function, we see that the corrupted binary vector now maps to the number 40. So, it seems as though a single bit-flip has completly changed the meaning of our encoding! Furthermore, there is no indication that anything has gone wrong. In this case, we say that the encoding has undergone a **logical error**. The error **distance** is the minimum **Hamming weight** of an error required to cause an undetectable logical error. Here we define the weight of the error as the number of non-zero entries. In the example above, our error vector had Hamming weight $1$, so the distance of the code is $d=1$.

### Exercise 1.02
Write a function (in `srx/hwbsc/binary.py`) that computes the Hamming weight of a binary numpy array.
 - As always, be sure to fully document and test 

## The goal of error correction

The ultimate aim of an error correction protocol is to find an alterantive encoding for binary strings that has a larger error distance. In practice, this is achieved by adding redundancy to the system by encoding using more bits than the raw encoding. The simplest manifestation of this philosophy is the **repetition code** where each bit is duplicated $n$ times.

For example, the redundant encoding of the binary number ${\text{bin}}(42)$ is given by:

In [20]:
binary42 = np.array([1, 0, 1, 0, 1, 0], dtype=int)
binary42_rep3_encoding = np.array([[1,1,1], [0,0,0], [1,1,1], [0,0,0], [1,1,1], [0,0,0]], dtype=int)

**Resilience against single-bit errors** 

In the above, we see that each bit in the original encoding has been duplicated three times. We now use three times the number of bits, but this affords us the ability to detect when error occurs. Eg. imagine that our encoded vector is subject to a weight one error:

In [21]:
error = np.zeros(binary42_rep3_encoding.shape, dtype=int)
error[0,0] = 1
print(error)

[[1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [22]:
Errored_binary42_rep3_encoding = (binary42_rep3_encoding + error) %2
print(Errored_binary42_rep3_encoding)

[[0 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]]


Now, in the above, we see that an error has been applied to the first bit in the first block of qubits. This block now reads $[0,1,1]$. By performing a majority vote, we can correct this block back to its correct form $[1,1,1]$.

**Resilience against 2-bit errors**

Ok, so lets see how it deals with a 2-bit error in the same block:

In [23]:
error = np.zeros(binary42_rep3_encoding.shape, dtype=int)
error[0,0] = 1
error[0,1] =1
print(error)

[[1 1 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [24]:
Errored_binary42_rep3_encoding = (binary42_rep3_encoding + error) %2
print(Errored_binary42_rep3_encoding)

[[0 0 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]]


Now, we see that there is something wrong in the first block. If we perform the majority vote, the block is corrected from $[0,0,1]$ to $[0,0,0]$. This is of course the wrong encoding, so our logical information has been corrupted. However, we do not say that a weight-2 error like this is a logical error, as we were able to detect that something was wrong (albeit not able to correctly decode it).

**Resilience to weight-3 errors**

Ok, so now what happens if we have a weight-3 erorr on the first block.



In [25]:
error = np.zeros(binary42_rep3_encoding.shape, dtype=int)
error[0,0] = 1
error[0,1] =1
error[0,2] = 1
print(error)


[[1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [26]:
Errored_binary42_rep3_encoding = (binary42_rep3_encoding + error) %2
print(Errored_binary42_rep3_encoding)

[[0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]]


In this case, the first block reads $[0,0,0]$. It looks like a completly normal part of the 3-bit repetition code encoding! In this case, we have an undetected error that has changed the meaning of our encoding. Ie. a logical error. The three-bit encoding therefore has a distance of $d=3$. 

### Exercise 1.03
Write a function that encodes any binary string into an $n$-bit repetition code.
 - See the skeleton in `src/hwbsc/repetition_code.py`
 - Write tests in `tests/repetition_code.py`
 - Docstrings as always

### Exercise 1.04

Write a majority vote decoder for a repetition code of length $n$.
 - See the skeleton in `src/hwbsc/repetition_code.py`
 - Write tests in `tests/repetition_code.py`
 - Docstrings as always

## The distance of repetition code

### Exercise 1.05

The distance of an $n$-bit repetition code is given by $d=n$.

  - Verify the above statement using your new encoding and decoding functions.
  - In general, a distance $d$ code can exactly correct $(d-1)/2$ errors. Verify this statement using your encoding and decoding function for a range of repetition code lengths.
  - If you have time (and motivation) write an an analytical proof of the above.